In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# Doing Run3?
ISRUN3 = False

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

BNB = ""
EXT = ""
NU  = ""
NUE = ""
DRT = ""
NCPI0 = ""
CCPI0 = ""
CCNOPI = ""
NCCPI = ""
NCNOPI = ""

if ISRUN3:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_G1_beam_good_reco2_1e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G1_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    NCPI0 = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2'
    CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_run3_reco2_reco2'
    NCCPI = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_New_run3_reco2_reco2'
    NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_new_run3_reco2_reco2'
    
else:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    NCPI0 = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2'
    CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    NCCPI = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    
mc = uproot.open(ls.ntuple_path+NU+".root")[fold][tree]
ncpi0 = uproot.open(ls.ntuple_path+NCPI0+".root")[fold][tree]
ccpi0 = uproot.open(ls.ntuple_path+CCPI0+".root")[fold][tree]
ccnopi = uproot.open(ls.ntuple_path+CCNOPI+".root")[fold][tree]
nccpi = uproot.open(ls.ntuple_path+NCCPI+".root")[fold][tree]
ncnopi = uproot.open(ls.ntuple_path+NCNOPI+".root")[fold][tree]
nue = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]
data = uproot.open(ls.ntuple_path+BNB+".root")[fold][tree]
ext = uproot.open(ls.ntuple_path+EXT+".root")[fold][tree]
dirt = uproot.open(ls.ntuple_path+DRT+".root")[fold][tree]
lee = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]

uproot_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,ext,data,dirt]
#uproot_v = [lee,mc,ncpi0,ccpi0,nue,ext,data,dirt]

variables = [
    "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "category", 
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "NeutrinoEnergy2",
    "trk_llr_pid_score_v", # trk-PID score
    #"pi0_energy2_Y", # pi0 tagger variables
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", 
    "hits_y", "trk_phi", "trk_theta",
    "run", "sub", "evt",
    "nproton",
    "interaction","topological_score",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", 
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2"
]
#make the list unique
variables = list(set(variables))
print(variables)
    
WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune"]#, "weightsGenie", "weightsFlux", "weightsReint"]
WEIGHTSLEE = ["weightSpline","weightTune","weightSplineTimesTune", "leeweight"]#, "weightsGenie", "weightsFlux", "weightsReint"]
MCFVARS = ["mcf_nu_e","mcf_lep_e","mcf_actvol","mcf_nmm","mcf_nmp","mcf_nem","mcf_nep","mcf_np0","mcf_npp",
           "mcf_npm","mcf_mcshr_elec_etot","mcf_pass_ccpi0","mcf_pass_ncpi0",
           "mcf_pass_ccnopi","mcf_pass_ncnopi","mcf_pass_cccpi","mcf_pass_nccpi"]
SYSTEMATICS = ['weightsFlux','weightsGenie']

nue = nue.pandas.df(variables + WEIGHTS + SYSTEMATICS, flatten=False)
mc = mc.pandas.df(variables + WEIGHTS + MCFVARS  + SYSTEMATICS, flatten=False)
ncpi0 = ncpi0.pandas.df(variables + WEIGHTS  + SYSTEMATICS, flatten=False)
ccpi0 = ccpi0.pandas.df(variables + WEIGHTS  + SYSTEMATICS, flatten=False)
ccnopi = ccnopi.pandas.df(variables + WEIGHTS, flatten=False)
nccpi = nccpi.pandas.df(variables + WEIGHTS, flatten=False)
ncnopi = ncnopi.pandas.df(variables + WEIGHTS, flatten=False)
data = data.pandas.df(variables, flatten=False)
ext = ext.pandas.df(variables, flatten=False)
dirt = dirt.pandas.df(variables + WEIGHTS + SYSTEMATICS, flatten=False)
lee = lee.pandas.df(variables + WEIGHTSLEE + SYSTEMATICS, flatten=False)

In [ ]:
# load XSEC splines
spline_energy_v = []
spline_xsec_v = []
fspline = uproot.open("/home/david/Downloads/"+"ccqe_spline_ratios.root")
graph = fspline["nu_mu_ccqe_v304a"]
print (graph)
print (graph.npoints)
xspline = graph.xvalues
yspline = graph.yvalues
for n in range(len(xspline)):
    if (n%100 == 0):
        spline_energy_v.append(xspline[n])
        spline_xsec_v.append(yspline[n])
print ("Energy Bins:")
print (spline_energy_v)

In [ ]:
print (spline_energy_v[1]-spline_energy_v[0])
print (spline_energy_v[6]-spline_energy_v[5])
print (spline_energy_v[501]-spline_energy_v[500])
SPLINEWIDTH = spline_energy_v[1]-spline_energy_v[0]

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
spline_interp_func = interp1d(spline_energy_v, spline_xsec_v)

In [ ]:
def spline_interp_func_offset(energy,offset):
    return spline_interp_func(energy+offset)
spline_interp_func_offset_v = np.vectorize(spline_interp_func_offset)

In [ ]:
def weight_ratio(energy,offset):
    return spline_interp_func_offset(energy,offset)/spline_interp_func_offset(energy,0.)
weight_ratio_v = np.vectorize(weight_ratio)

In [ ]:
erange_v = np.linspace(0.105,1.5,100)

In [ ]:
fig = plt.figure(figsize=(6,6))
#plt.plot(erange_v,spline_xsec_v)
plt.plot(erange_v,spline_interp_func(erange_v),color='r',label='CV')
plt.plot(erange_v,spline_interp_func_offset_v(erange_v,0.09),color='b',label='90 MeV')
#plt.plot(erange_v,spline_interp_func_offset_v(erange_v,-0.09),color='m',label='-90 MeV')
plt.xlim([0,1.5])
plt.legend(fontsize=16)
plt.grid()
plt.xlabel('neutrino energy [GeV]',fontsize=16)
plt.ylabel('xsec',fontsize=16)
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
#plt.plot(erange_v,spline_xsec_v)
#plt.plot(erange_v,spline_interp_func(erange_v),color='r',label='CV')
plt.plot(erange_v,weight_ratio_v(erange_v,0.09),color='b',label='90 MeV offset')
#plt.plot(erange_v,weight_ratio_v(erange_v,-0.09),color='m')
plt.xlim([0,1.5])
plt.legend(fontsize=16)
plt.grid()
plt.xlabel('neutrino energy [GeV]',fontsize=16)
plt.ylabel('xsec ratio',fontsize=16)
plt.ylim([0,5])
plt.show()

In [ ]:
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,dirt]

for i,df in enumerate(df_v):
    
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

In [ ]:
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,ext,data,dirt]


for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel

In [ ]:
# avoid double-counting of events out of FV in the NC/CC pi0 samples
# not needed anymore since we improved matching with filtered samples
#ncpi0 = ncpi0.query('category != 5')
#ccpi0 = ccpi0.query('category != 5')
#ccnopi = ccnopi.query('category != 5')
#nccpi = nccpi.query('category != 5')
#ncnopi = ncnopi.query('category != 5')

# avoid recycling unbiased ext events (i.e. selecting a slice with little nu content from these samples)
ccnopi = ccnopi.query('(nslice==0 | (slnunhits/slnhits)>0.2)')
nccpi = nccpi.query('(nslice==0 | (slnunhits/slnhits)>0.2)')
ncnopi = ncnopi.query('(nslice==0 | (slnunhits/slnhits)>0.2)')

# add back the cosmic category, for background only
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.2), 'category'] = 4

In [ ]:
lee["is_signal"] = lee["category"] == 11
data["is_signal"] = data["category"] == 11
nue["is_signal"] = nue["category"] == 11
mc["is_signal"] = mc["category"] == 11
dirt["is_signal"] = dirt["category"] == 11
ext["is_signal"] = ext["category"] == 11
ncpi0["is_signal"] = ncpi0["category"] == 11
ccpi0["is_signal"] = ccpi0["category"] == 11
ccnopi["is_signal"] = ccnopi["category"] == 11
nccpi["is_signal"] = nccpi["category"] == 11
ncnopi["is_signal"] = ncnopi["category"] == 11

lee.loc[lee['category'] == 1, 'category'] = 111
lee.loc[lee['category'] == 10, 'category'] = 111
lee.loc[lee['category'] == 11, 'category'] = 111

In [ ]:
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,dirt]

for i,df in enumerate(df_v):
    df.loc[ (df['interaction'] == 0)&(df['nu_pdg']==14)&(df['nu_e']<1.5)&(df['nu_e']>0.105)&(df['ccnc']==0), 'weightXSEC' ] = df['weightSpline'] *  weight_ratio_v(df['nu_e'],0.0)
    df.loc[ (df['interaction'] == 0)&(df['nu_pdg']==14)&(df['nu_e']<1.5)&(df['nu_e']>0.105)&(df['ccnc']==0), 'weightSplineTimesTune' ] = df['weightXSEC'] * df['weightTune']
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

In [ ]:
import plotter
import importlib
importlib.reload(plotter)

In [ ]:
samples = {
    "mc": mc,
    "nue": nue,
    "data": data,
    "ext": ext,
    "dirt": dirt,
    "ncpi0": ncpi0,
    "ccpi0": ccpi0,
    "ccnopi": ccnopi,
    "nccpi": nccpi,
    "ncnopi": ncnopi,
    "lee": lee
}


if ISRUN3:
    scaling = 101.0/0.865
else:
    scaling = 101.0/4.07
scaling = 1

SPLIT = 1.0

#''' 0109
if ISRUN3:
    weights = {
        "mc": 6.46e-3 * SPLIT * scaling,
        "nue": 1.19e-4 * SPLIT * scaling,
        "ext": 2.18e-2 * SPLIT * scaling,
        "dirt": 2.66e-2 * scaling,
        "lee": 1.19e-4 * SPLIT * scaling,
        "ncpi0": 3.79e-3 * SPLIT * scaling,
        "ccpi0": 1.34e-3 * SPLIT * scaling,
        "ccnopi": 3.11e-03 * SPLIT * scaling,
        "nccpi": 8.24e-04 * SPLIT * scaling,
        "ncnopi": 1.86e-03 * SPLIT * scaling,
    }
    pot = 0.865e19*scaling
else:
    weights = {
        "mc": 4.02e-2 * SPLIT * scaling,
        "nue": 7.72e-4 * SPLIT * scaling,
        "ext": 1.39e-1 * SPLIT * scaling,
        "dirt": 1.26e-1 * scaling,
        "lee": 7.72e-4 * SPLIT * scaling,
        "ncpi0": 1.77e-2 * SPLIT * scaling,
        "ccpi0": 1.17e-2 * SPLIT * scaling,
        "ccnopi": 1.37e-02 * SPLIT * scaling,
        "nccpi": 4.55e-03 * SPLIT * scaling,
        "ncnopi": 9.60e-03 * SPLIT * scaling,
    }
    pot = 4.07e19*scaling
#'''
my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)

In [ ]:
# muon selection
QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and trk_len > 20'
QUERY += ' and topological_score > 0.5'
QUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
QUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
QUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
QUERY += ' and trkpid > 0.2'

print (QUERY)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
VARIABLE, BINS, RANGE, XTIT = 'trk_len',25,(10,250),"Track Length [cm]"

fig, ax1, ax2 = my_plotter.plot_variable(
    VARIABLE,   
    query=QUERY,
    kind="event_category",
    #kind="interaction",
    #kind="sample",
    #kind='particle_pdg',
    draw_sys=True,
    title=XTIT,
    #bins=asymm_bins,
    bins=BINS,
    range=RANGE,
)

#print("Profile likelihood: %.2f sigma @ 1.32e21 POT" % my_plotter.significance_likelihood)
#print("s/sqrt(b): %.2f sigma @ 1.32e21 POT" % my_plotter.significance)

#ax1.set_ylim(0,1200)
#ax1.set_yscale("log")
#ax1.set_ylim(0,0.25)
ax2.set_ylim(0.5,1.5)
ax1.set_ylim(0,400)
#ax1.set_ylim(0.,ax1.get_ylim()[1]*1.5)
fig.savefig(ls.plots_path+VARIABLE+"_"+date_time+"_000MeV.pdf")